# Different ratings

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

## Test for a single link

In [2]:
URL ="https://www.indeed.com/cmp/Amazon.com/reviews?fjobtitle=Software+Engineer&ftopic=wlbalance"

In [3]:
#conducting a request of the stated URL above:
page = requests.get(URL)
#specifying a desired format of “page” using the html parser - this allows python to read the various components of the page, rather than treating it as one long string.
soup = BeautifulSoup(page.text, "html.parser")
#printing soup in a more structured tree format that makes for easier reading
#print(soup.prettify())

## Extract Ratings Sample

In [44]:
def extract_ratings_from_result(soup): 
    ratings = []
    spans = soup.findAll('div', attrs={'class': "cmp-ratingNumber"})
    for span in spans:
        ratings.append(float(span.text.strip()))
    return(ratings)

## Metadata for different kinds of companies

In [23]:
# metadata for software company : google, amazon, facebook, microsoft
max_results_per_job = 30
company_set_1 = ['amazon','google','facebook','microsoft']
job_set_1 = ['Software+Development+Intern','Software+Engineer','Senior+Software+Engineer','Software+Engineering+Manager']
# for eletrical company: intel, ibm, cisco, nasa
company_set_2 = ['intel','ibm','cisco','nasa']
job_set_2 = ['Network+Engineer','Design+Engineer','Electrical+Engineer','System+Engineer','Mechanical+Engineer']
# for data_related job: data-scientist data-engineer

ratings = ["wlbalance","paybenefits","jobsecadv","mgmt","culture"]
column = ["company","job_title", "wlbalance","paybenefits","jobsecadv","mgmt","culture"]
sample_df = pd.DataFrame(columns = column)

## Scraping data to dataframe

In [24]:
#scraping for IT company code: google, amazon, facebook, microsoft
for company in company_set_1:
    for job in job_set_1:
        post = []
        post.append(company)
        num = (len(sample_df) + 1) 
        post.append(job) 
        for rating in ratings:
            li = []
            for start in range(0, max_results_per_job, 10):
                page = requests.get('https://www.indeed.com/cmp/' + str(company) + '/reviews?fjobtitle='+ str(job) + '&ftopic=' + str(rating) + '&start=' + str(start))
            #page = requests.get(‘http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=' + str(city) + ‘&start=’ + str(start))
                time.sleep(1)  #ensuring at least 1 second between page grabs
                soup = BeautifulSoup(page.text, "html.parser")
                #specifying row num for index of job posting in dataframe
                spans = soup.findAll('div', attrs={'class': "cmp-ratingNumber"})
                for span in spans:
                    li.append(float(span.text.strip()))
            if len(li)==0:
                post.append('Nan')
            else:
                post.append(sum(li) / len(li))
        sample_df.loc[num] = post                                                                                                        

In [ ]:
# scraping for electrical company: intel, ibm, cisco, nasa
for company in company_set_2:
    for job in job_set_1:
        post = []
        post.append(company)
        post.append(job) 
        num = (len(sample_df) + 1) 
        for rating in ratings:
            li = []
            for start in range(0, max_results_per_job, 10):
                page = requests.get('https://www.indeed.com/cmp/' + str(company) + '/reviews?fjobtitle='+ str(job) + '&ftopic=' + str(rating) + '&start=' + str(start))
            #page = requests.get(‘http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=' + str(city) + ‘&start=’ + str(start))
                time.sleep(1)  #ensuring at least 1 second between page grabs
                soup = BeautifulSoup(page.text, "html.parser")
                #specifying row num for index of job posting in dataframe
                spans = soup.findAll('div', attrs={'class': "cmp-ratingNumber"})
                for span in spans:
                    li.append(float(span.text.strip()))
            post.append(sum(li) / len(li) )
        sample_df.loc[num] = post                                                                                                        

In [25]:
sample_df

,company,job_title,wlbalance,paybenefits,jobsecadv,mgmt,culture
1,amazon,Software+Development+Intern,4.08333,3.5,4.05882,3.75,4.16667
2,amazon,Software+Engineer,3.53333,3.77778,3.53968,2.61364,3.26923
3,amazon,Senior+Software+Engineer,3.85714,3.33333,3.28571,2.83333,3.1
4,amazon,Software+Engineering+Manager,3.5,3.66667,3.54545,2.88889,3.54545
5,google,Software+Development+Intern,Nan,5,5,Nan,Nan
6,google,Software+Engineer,4.33333,3.7,4.28571,3.625,3.7
7,google,Senior+Software+Engineer,3.875,3.66667,3.5,3.6,4.125
8,google,Software+Engineering+Manager,3,Nan,5,Nan,5
9,facebook,Software+Development+Intern,Nan,Nan,Nan,Nan,Nan
10,facebook,Software+Engineer,4,Nan,4.33333,4,4.66667


## Download CSV file

In [ ]:
#saving sample_df as a local csv file — define your own local path to save contents 
sample_df.to_csv("ratings.csv", encoding=’utf-8')